In [1]:
import pandas as pd
import os
import lightgbm as lgb
import xgboost as xgb
import numpy as np
import matplotlib.pyplot as plt
import time

from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import Lasso
from sklearn.ensemble import GradientBoostingRegressor

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import ModelCheckpoint
import pickle

Using TensorFlow backend.
C:\Users\dl\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\dl\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\dl\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\dl\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: Future

In [2]:
price = pd.read_csv('./transpose-123 - transpose.csv')
path = 'pepper_supply/pepper'
o00 = pd.read_excel('./{}_00.xlsx'.format(path))
o01 = pd.read_excel('./{}_01.xlsx'.format(path))
o02 = pd.read_excel('./{}_02.xlsx'.format(path))
o03 = pd.read_excel('./{}_03.xlsx'.format(path))
o04 = pd.read_excel('./{}_04.xlsx'.format(path))
o05 = pd.read_excel('./{}_05.xlsx'.format(path))
o06 = pd.read_excel('./{}_06.xlsx'.format(path))
o07 = pd.read_excel('./{}_07.xlsx'.format(path))
o08 = pd.read_excel('./{}_08.xlsx'.format(path))
o09 = pd.read_excel('./{}_09.xlsx'.format(path))
o10 = pd.read_excel('./{}_10.xlsx'.format(path))
o11 = pd.read_excel('./{}_11.xlsx'.format(path))
o12 = pd.read_excel('./{}_12.xlsx'.format(path))
o13 = pd.read_excel('./{}_13.xlsx'.format(path))
o14 = pd.read_excel('./{}_14.xlsx'.format(path))
o15 = pd.read_excel('./{}_15.xlsx'.format(path))
o16 = pd.read_excel('./{}_16.xlsx'.format(path))
o17 = pd.read_excel('./{}_17.xlsx'.format(path))
o18 = pd.read_excel('./{}_18.xlsx'.format(path))
o19 = pd.read_excel('./{}_19.xlsx'.format(path))
# o20 = pd.read_excel('./{}_20.xlsx'.format(path))

supply = pd.concat([o00, o01, o02, o03, o04, o05, o06, o07, o08, o09, o10, o11, o12, o13, o14, o15, o16, o17, o18, o19])
# supply = supply[2:]
supply['년도'] = supply['거래년월일'].apply(lambda x: str(x)[:4]).astype(str)
supply['월'] = supply['거래년월일'].apply(lambda x: str(x)[4:6]).astype(str)
supply['일'] = supply['거래년월일'].apply(lambda x: str(x)[6:8]).astype(str)
supply['거래량'] = supply['거래량'].apply(lambda x: ''.join(str(x).split(','))).astype(int)
supply

,거래년월일,품목명,시장코드,시장명,출하지코드,출하지명,거래량,품목코드,년도,월,일
0,총 합계,전체,NaN,NaN,NaN,NaN,79211,NaN,총 합계,,
1,소계,건고추(일반),NaN,NaN,NaN,NaN,79211,120701.0,소계,,
2,20001230,건고추(일반),210001.0,부산엄궁도매시장,356010,충청남도서산시동문동,5100,120701.0,2000,12,30
3,20001229,건고추(일반),311201.0,구리도매시장,501000,광주동구,60,120701.0,2000,12,29
4,20001229,건고추(일반),210001.0,부산엄궁도매시장,621010,경상남도김해시부원동,4500,120701.0,2000,12,29
...,...,...,...,...,...,...,...,...,...,...,...
2129,20190110,건고추(일반),230001.0,인천구월도매시장,800CHN,중국,120,120701.0,2019,01,10
2130,20190108,건고추(일반),380201.0,울산도매시장,800000,수입산,40,120701.0,2019,01,08
2131,20190105,건고추(일반),360301.0,순천도매시장,548840,전라남도고흥군남양면,36,120701.0,2019,01,05
2132,20190105,건고추(일반),360301.0,순천도매시장,548800,전라남도고흥군고흥읍,36,120701.0,2019,01,05


In [3]:
price['년도'] = price['년도'].apply(lambda x: str(x))
price['월'] = price['월/일'].apply(lambda x: str(x)[0:2]).astype(str)
price['일'] = price['월/일'].apply(lambda x: str(x)[3:5]).astype(str)
price

,년도,월/일,평균,월,일
0,2000,01/03,"103,000",01,03
1,2000,01/04,"97,000",01,04
2,2000,01/05,"95,000",01,05
3,2000,01/06,"98,000",01,06
4,2000,01/07,"98,000",01,07
...,...,...,...,...,...
5282,2020,06/01,"61,000",06,01
5283,2020,06/02,"59,800",06,02
5284,2020,06/03,"60,600",06,03
5285,2020,06/04,"57,400",06,04


In [4]:
sup_total = supply.groupby(['년도','월','일'], as_index=False).sum()
sup_total = sup_total[:-2]
sup_total

,년도,월,일,시장코드,거래량,품목코드
0,2000,01,03,380201.0,130,120701.0
1,2000,01,06,380201.0,130,120701.0
2,2000,01,12,380201.0,130,120701.0
3,2000,01,14,380201.0,130,120701.0
4,2000,01,15,380201.0,130,120701.0
...,...,...,...,...,...,...
5012,2019,12,26,1210615.0,282,482804.0
5013,2019,12,27,690011.0,38,362103.0
5014,2019,12,28,830306.0,190,362103.0
5015,2019,12,30,530210.0,264,241402.0


In [5]:
df = pd.merge(price, sup_total, on=['년도','월','일'])
df

,년도,월/일,평균,월,일,시장코드,거래량,품목코드
0,2000,01/03,"103,000",01,03,380201.0,130,120701.0
1,2000,01/06,"98,000",01,06,380201.0,130,120701.0
2,2000,01/12,"97,000",01,12,380201.0,130,120701.0
3,2000,01/14,"95,000",01,14,380201.0,130,120701.0
4,2000,01/15,"91,000",01,15,380201.0,130,120701.0
...,...,...,...,...,...,...,...,...
4204,2019,12/24,"65,000",12,24,690017.0,600,362103.0
4205,2019,12/26,"64,000",12,26,1210615.0,282,482804.0
4206,2019,12/27,"64,200",12,27,690011.0,38,362103.0
4207,2019,12/30,"65,200",12,30,530210.0,264,241402.0


In [6]:
df2 = df[['년도','월','일','평균','거래량']]
df2['년도'] = df2['년도'].apply(lambda x: int(x))
df2['월'] = df2['월'].apply(lambda x: int(x))
df2['일'] = df2['일'].apply(lambda x: int(x))
df2['평균'] = df2['평균'].apply(lambda x: ''.join(str(x).split(','))).astype(int) 

df2

C:\Users\dl\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\dl\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\dl\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

,년도,월,일,평균,거래량
0,2000,1,3,103000,130
1,2000,1,6,98000,130
2,2000,1,12,97000,130
3,2000,1,14,95000,130
4,2000,1,15,91000,130
...,...,...,...,...,...
4204,2019,12,24,65000,600
4205,2019,12,26,64000,282
4206,2019,12,27,64200,38
4207,2019,12,30,65200,264


In [7]:
a = list(df2['평균'])
df2['d+1'] = a
a.insert(0,0)
a = a[:-1]

df2['d-0'] = a
a.insert(0,0)
a = a[:-1]

df2['d-1'] = a
a.insert(0,0)
a = a[:-1]

df2['d-2'] = a
a.insert(0,0)
a = a[:-1]

df2['d-3'] = a
a.insert(0,0)
a = a[:-1]

df2['d-4'] = a
a.insert(0,0)
a = a[:-1]

df2['d-5'] = a
a.insert(0,0)
a = a[:-1]

df2['d-6'] = a
a.insert(0,0)
a = a[:-1]

df2['d-7'] = a
df2.head(10)

C:\Users\dl\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\dl\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,년도,월,일,평균,거래량,d+1,d-0,d-1,d-2,d-3,d-4,d-5,d-6,d-7
0,2000,1,3,103000,130,103000,0,0,0,0,0,0,0,0
1,2000,1,6,98000,130,98000,103000,0,0,0,0,0,0,0
2,2000,1,12,97000,130,97000,98000,103000,0,0,0,0,0,0
3,2000,1,14,95000,130,95000,97000,98000,103000,0,0,0,0,0
4,2000,1,15,91000,130,91000,95000,97000,98000,103000,0,0,0,0
5,2000,1,25,95000,130,95000,91000,95000,97000,98000,103000,0,0,0
6,2000,1,27,95000,130,95000,95000,91000,95000,97000,98000,103000,0,0
7,2000,1,28,95000,130,95000,95000,95000,91000,95000,97000,98000,103000,0
8,2000,1,31,95000,130,95000,95000,95000,95000,91000,95000,97000,98000,103000
9,2000,2,8,90000,489,90000,95000,95000,95000,95000,91000,95000,97000,98000


In [8]:
del df2['평균']
df2 = df2[8:]
df2 = df2.reset_index(drop=True)
df2

,년도,월,일,거래량,d+1,d-0,d-1,d-2,d-3,d-4,d-5,d-6,d-7
0,2000,1,31,130,95000,95000,95000,95000,91000,95000,97000,98000,103000
1,2000,2,8,489,90000,95000,95000,95000,95000,91000,95000,97000,98000
2,2000,2,12,130,64000,90000,95000,95000,95000,95000,91000,95000,97000
3,2000,2,14,130,57000,64000,90000,95000,95000,95000,95000,91000,95000
4,2000,2,19,720,40000,57000,64000,90000,95000,95000,95000,95000,91000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4196,2019,12,24,600,65000,65000,61800,70000,69000,68000,71400,71800,72000
4197,2019,12,26,282,64000,65000,65000,61800,70000,69000,68000,71400,71800
4198,2019,12,27,38,64200,64000,65000,65000,61800,70000,69000,68000,71400
4199,2019,12,30,264,65200,64200,64000,65000,65000,61800,70000,69000,68000


In [9]:
y = df2['d+1']
del df2['d+1']

In [10]:
model = Sequential()
model.add(Dense(30, input_dim=12, activation='relu'))
model.add(Dense(900, activation='relu'))
model.add(Dense(4096, activation='relu'))
model.add(Dense(1028, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(2, activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error',
             optimizer='adam',
             metrics=['accuracy'])


seed_pred_result = []
rmse_result = []
for i in range(10):
    rand_num = i
    x_train, x_test,y_train, y_test = train_test_split(df2, y, test_size=0.2, random_state=rand_num)

    gboost = GradientBoostingRegressor(random_state=rand_num)
    xgboost = xgb.XGBRegressor(random_state=rand_num)
    lightgbm = lgb.LGBMRegressor(random_state=rand_num)
    lasso = Lasso(alpha=1.0, random_state=rand_num)

    models = [{'model':xgboost, 'name':'XGBoost'},
              {'model':gboost, 'name': 'GradientBoosting'},
              {'model':lightgbm, 'name':'LightGBM'},
              {'model':lasso, 'name': 'Lasso'}]
    result = []
    rmse = []
    result.append(y_test)

    for m in models:
        m['model'].fit(x_train.values, y_train)
        pickle.dump(m['model'], open("pima_{}_{}.pickle.dat".format(i, m['name']), "wb"))
        #.save_model('./{}_model'.format(m['name']))

    for m in models:
    #         pickle.dump(m['model'], open("pima_{}.pickle.dat".format(m['name']), "wb"))
        loaded_model = pickle.load(open("pima_{}_{}.pickle.dat".format(i, m['name']), "rb"))
    #     loaded_model.predict(X_test)
        predictions = loaded_model.predict(x_test.values) 
        result.append(predictions)
        rmse.append({m['name']:sqrt(mean_squared_error(y_test, predictions))})

    
    # 모델 저장 폴더 설정
    MODEL_DIR = './model_{}/'.format(i)
    if not os.path.exists(MODEL_DIR):
        os.mkdir(MODEL_DIR)
        
    # 모델 저장 조건 설정
    modelpath = MODEL_DIR + 'keras_{epoch:02d}-{val_loss:.4f}.hdf5'
    checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
    history = model.fit(x_train, y_train, validation_split=0.2, epochs=100, batch_size=128, callbacks=[checkpointer])
    
    modelPred = model.predict(x_test)
    Y_prediction = modelPred.flatten()

    result.append(Y_prediction)
    rmse.append({'keras':sqrt(mean_squared_error(y_test, modelPred))})

    dict = {'y': result[0], 'xgboost': result[1], 'gboost': result[2], 'lightgbm':result[3], 'lasso':result[4],'keras': result[5]}

    df_res = pd.DataFrame(dict)
    df_res['average'] = df_res[['xgboost', 'gboost', 'lightgbm', 'lasso', 'keras']].apply(lambda x: round(x.mean()), axis=1)
    rmse.append({'average':sqrt(mean_squared_error(y_test, df_res['average']))})

    rmse_result.append(rmse)
    seed_pred_result.append(df_res)    

C:\Users\dl\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 37493824159.96366, tolerance: 393232943.724615
  positive)



Train on 2688 samples, validate on 672 samples
Epoch 1/100
2688/2688 [==============================] - 2s 849us/step - loss: 4806947535.2381 - accuracy: 0.0000e+00 - val_loss: 4677829241.9048 - val_accuracy: 0.0000e+00

Epoch 00001: val_loss improved from inf to 4677829241.90476, saving model to ./model_0/keras_01-4677829241.9048.hdf5
Epoch 2/100
2688/2688 [==============================] - 2s 726us/step - loss: 4700381208.3810 - accuracy: 0.0000e+00 - val_loss: 4640579730.2857 - val_accuracy: 0.0000e+00

Epoch 00002: val_loss improved from 4677829241.90476 to 4640579730.28571, saving model to ./model_0/keras_02-4640579730.2857.hdf5
Epoch 3/100
2688/2688 [==============================] - 2s 722us/step - loss: 4692563577.9048 - accuracy: 0.0000e+00 - val_loss: 4640344259.0476 - val_accuracy: 0.0000e+00

Epoch 00003: val_loss improved from 4640579730.28571 to 4640344259.04762, saving model to ./model_0/keras_03-4640344259.0476.hdf5
Epoch 4/100
2688/2688 [==============================

2688/2688 [==============================] - 2s 704us/step - loss: 4691899721.1429 - accuracy: 0.0000e+00 - val_loss: 4640285110.8571 - val_accuracy: 0.0000e+00

Epoch 00027: val_loss improved from 4640287110.09524 to 4640285110.85714, saving model to ./model_0/keras_27-4640285110.8571.hdf5
Epoch 28/100
2688/2688 [==============================] - 2s 706us/step - loss: 4691897185.5238 - accuracy: 0.0000e+00 - val_loss: 4640282282.6667 - val_accuracy: 0.0000e+00

Epoch 00028: val_loss improved from 4640285110.85714 to 4640282282.66667, saving model to ./model_0/keras_28-4640282282.6667.hdf5
Epoch 29/100
2688/2688 [==============================] - 2s 704us/step - loss: 4691894662.0952 - accuracy: 0.0000e+00 - val_loss: 4640279893.3333 - val_accuracy: 0.0000e+00

Epoch 00029: val_loss improved from 4640282282.66667 to 4640279893.33333, saving model to ./model_0/keras_29-4640279893.3333.hdf5
Epoch 30/100
2688/2688 [==============================] - 2s 710us/step - loss: 4691892406.8571 - 

Epoch 54/100
2688/2688 [==============================] - 2s 715us/step - loss: 4691832563.8095 - accuracy: 0.0000e+00 - val_loss: 4640218160.7619 - val_accuracy: 0.0000e+00

Epoch 00054: val_loss improved from 4640220476.95238 to 4640218160.76190, saving model to ./model_0/keras_54-4640218160.7619.hdf5
Epoch 55/100
2688/2688 [==============================] - 2s 717us/step - loss: 4691830016.0000 - accuracy: 0.0000e+00 - val_loss: 4640215576.3810 - val_accuracy: 0.0000e+00

Epoch 00055: val_loss improved from 4640218160.76190 to 4640215576.38095, saving model to ./model_0/keras_55-4640215576.3810.hdf5
Epoch 56/100
2688/2688 [==============================] - 2s 720us/step - loss: 4691827504.7619 - accuracy: 0.0000e+00 - val_loss: 4640212918.8571 - val_accuracy: 0.0000e+00

Epoch 00056: val_loss improved from 4640215576.38095 to 4640212918.85714, saving model to ./model_0/keras_56-4640212918.8571.hdf5
Epoch 57/100
2688/2688 [==============================] - 2s 718us/step - loss: 46918

Epoch 81/100
2688/2688 [==============================] - 2s 708us/step - loss: 4691765260.1905 - accuracy: 0.0000e+00 - val_loss: 4640151478.8571 - val_accuracy: 0.0000e+00

Epoch 00081: val_loss improved from 4640153648.76190 to 4640151478.85714, saving model to ./model_0/keras_81-4640151478.8571.hdf5
Epoch 82/100
2688/2688 [==============================] - 2s 715us/step - loss: 4691762651.4286 - accuracy: 0.0000e+00 - val_loss: 4640148992.0000 - val_accuracy: 0.0000e+00

Epoch 00082: val_loss improved from 4640151478.85714 to 4640148992.00000, saving model to ./model_0/keras_82-4640148992.0000.hdf5
Epoch 83/100
2688/2688 [==============================] - 2s 712us/step - loss: 4691760310.8571 - accuracy: 0.0000e+00 - val_loss: 4640146432.0000 - val_accuracy: 0.0000e+00

Epoch 00083: val_loss improved from 4640148992.00000 to 4640146432.00000, saving model to ./model_0/keras_83-4640146432.0000.hdf5
Epoch 84/100
2688/2688 [==============================] - 2s 714us/step - loss: 46917

...

C:\Users\dl\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 37891657755.36702, tolerance: 392676717.69531894
  positive)


2688/2688 [==============================] - 2s 715us/step - loss: 4687785788.9524 - accuracy: 0.0000e+00 - val_loss: 4658592938.6667 - val_accuracy: 0.0000e+00

Epoch 00001: val_loss improved from inf to 4658592938.66667, saving model to ./model_9/keras_01-4658592938.6667.hdf5
Epoch 2/100
2688/2688 [==============================] - 2s 712us/step - loss: 4687783204.5714 - accuracy: 0.0000e+00 - val_loss: 4658590086.0952 - val_accuracy: 0.0000e+00

Epoch 00002: val_loss improved from 4658592938.66667 to 4658590086.09524, saving model to ./model_9/keras_02-4658590086.0952.hdf5
Epoch 3/100
2688/2688 [==============================] - 2s 713us/step - loss: 4687780583.6190 - accuracy: 0.0000e+00 - val_loss: 4658587940.5714 - val_accuracy: 0.0000e+00

Epoch 00003: val_loss improved from 4658590086.09524 to 4658587940.57143, saving model to ./model_9/keras_03-4658587940.5714.hdf5
Epoch 4/100
2688/2688 [==============================] - 2s 710us/step - loss: 4687778340.5714 - accuracy: 0.0000

Epoch 28/100
2688/2688 [==============================] - 2s 712us/step - loss: 4687718643.8095 - accuracy: 0.0000e+00 - val_loss: 4658525476.5714 - val_accuracy: 0.0000e+00

Epoch 00028: val_loss improved from 4658528109.71429 to 4658525476.57143, saving model to ./model_9/keras_28-4658525476.5714.hdf5
Epoch 29/100
2688/2688 [==============================] - 2s 713us/step - loss: 4687716181.3333 - accuracy: 0.0000e+00 - val_loss: 4658523184.7619 - val_accuracy: 0.0000e+00

Epoch 00029: val_loss improved from 4658525476.57143 to 4658523184.76190, saving model to ./model_9/keras_29-4658523184.7619.hdf5
Epoch 30/100
2688/2688 [==============================] - 2s 709us/step - loss: 4687713840.7619 - accuracy: 0.0000e+00 - val_loss: 4658520332.1905 - val_accuracy: 0.0000e+00

Epoch 00030: val_loss improved from 4658523184.76190 to 4658520332.19048, saving model to ./model_9/keras_30-4658520332.1905.hdf5
Epoch 31/100
2688/2688 [==============================] - 2s 714us/step - loss: 46877

Epoch 55/100
2688/2688 [==============================] - 2s 707us/step - loss: 4687651523.0476 - accuracy: 0.0000e+00 - val_loss: 4658458477.7143 - val_accuracy: 0.0000e+00

Epoch 00055: val_loss improved from 4658460745.14286 to 4658458477.71429, saving model to ./model_9/keras_55-4658458477.7143.hdf5
Epoch 56/100
2688/2688 [==============================] - 2s 711us/step - loss: 4687649036.1905 - accuracy: 0.0000e+00 - val_loss: 4658456015.2381 - val_accuracy: 0.0000e+00

Epoch 00056: val_loss improved from 4658458477.71429 to 4658456015.23810, saving model to ./model_9/keras_56-4658456015.2381.hdf5
Epoch 57/100
2688/2688 [==============================] - 2s 712us/step - loss: 4687646720.0000 - accuracy: 0.0000e+00 - val_loss: 4658453504.0000 - val_accuracy: 0.0000e+00

Epoch 00057: val_loss improved from 4658456015.23810 to 4658453504.00000, saving model to ./model_9/keras_57-4658453504.0000.hdf5
Epoch 58/100
2688/2688 [==============================] - 2s 705us/step - loss: 46876

Epoch 82/100
2688/2688 [==============================] - 2s 707us/step - loss: 4687584377.9048 - accuracy: 0.0000e+00 - val_loss: 4658391088.7619 - val_accuracy: 0.0000e+00

Epoch 00082: val_loss improved from 4658393258.66667 to 4658391088.76190, saving model to ./model_9/keras_82-4658391088.7619.hdf5
Epoch 83/100
2688/2688 [==============================] - 2s 714us/step - loss: 4687582073.9048 - accuracy: 0.0000e+00 - val_loss: 4658388894.4762 - val_accuracy: 0.0000e+00

Epoch 00083: val_loss improved from 4658391088.76190 to 4658388894.47619, saving model to ./model_9/keras_83-4658388894.4762.hdf5
Epoch 84/100
2688/2688 [==============================] - 2s 710us/step - loss: 4687579404.1905 - accuracy: 0.0000e+00 - val_loss: 4658385944.3810 - val_accuracy: 0.0000e+00

Epoch 00084: val_loss improved from 4658388894.47619 to 4658385944.38095, saving model to ./model_9/keras_84-4658385944.3810.hdf5
Epoch 85/100
2688/2688 [==============================] - 2s 709us/step - loss: 46875

In [11]:
rmse_result  

[[{'XGBoost': 4770.638264530982},
  {'GradientBoosting': 4525.173802537123},
  {'LightGBM': 4481.2847548008185},
  {'Lasso': 4262.037756786643},
  {'keras': 66754.30203679144},
  {'average': 13894.95762393635}],
 [{'XGBoost': 5265.143389404209},
  {'GradientBoosting': 5000.246392497806},
  {'LightGBM': 5078.452497731403},
  {'Lasso': 4848.225386384527},
  {'keras': 70052.8249418304},
  {'average': 15023.239933868548}],
 [{'XGBoost': 4442.619188052273},
  {'GradientBoosting': 4341.259219553394},
  {'LightGBM': 4213.810640487963},
  {'Lasso': 4302.773864024105},
  {'keras': 66696.08136379643},
  {'average': 14117.918255802686}],
 [{'XGBoost': 5203.622467290613},
  {'GradientBoosting': 5082.822220862741},
  {'LightGBM': 5064.41011311647},
  {'Lasso': 4944.660761044388},
  {'keras': 67637.20677876698},
  {'average': 14522.938652512354}],
 [{'XGBoost': 4519.787515497257},
  {'GradientBoosting': 4329.727965449074},
  {'LightGBM': 4382.709279582772},
  {'Lasso': 4219.838267450731},
  {'keras'

In [12]:
seed_pred_result[0].head(10)

,y,xgboost,gboost,lightgbm,lasso,keras,average
3037,78000,78204.476562,79284.675653,80107.015208,79554.303611,2.097415,63431
3485,116000,104217.210938,109495.804934,112003.962968,105852.175838,2.097415,86314
3347,78000,74096.890625,75747.552446,74002.813661,76049.000200,2.097415,59980
72,34400,33868.429688,32766.122858,33189.689864,34303.230290,2.097415,26826
2284,39000,41748.621094,40249.208937,39034.043683,39705.019643,2.097415,32148
2651,65200,66127.601562,67104.592309,67450.023827,69062.149818,2.097415,53949
799,64200,62432.730469,62140.813228,62647.160116,60897.970053,2.097415,49624
2760,27000,34606.910156,35151.100218,35048.069177,32885.108931,2.097415,27539
1760,42400,39919.875000,40961.985136,41019.521718,40900.156448,2.097415,32561
1410,42800,40921.812500,40173.242066,40384.307575,42302.943881,2.097415,32757


In [13]:
seed_pred_result[1].head(10)

,y,xgboost,gboost,lightgbm,lasso,keras,average
3604,51400,54227.792969,55079.191771,55734.413396,53499.030399,4.190105,43709
781,53200,56590.945312,56704.321995,56936.408659,56926.737251,4.190105,45433
2165,161000,157694.156250,157465.984139,157639.294412,153873.974558,4.190105,125336
1217,101600,102894.671875,103190.286174,99506.271726,103282.249010,4.190105,81776
1006,72400,70366.093750,69301.267020,70378.808979,69038.263868,4.190105,55818
941,45200,45277.906250,43792.922506,45540.040701,46571.618650,4.190105,36237
2589,80000,74476.101562,74263.012945,77963.515158,73954.518020,4.190105,60132
2801,53200,51096.726562,51489.742170,51443.925752,50489.972865,4.190105,40905
2871,84000,80455.937500,82093.402257,83057.578562,81269.799915,4.190105,65376
1333,44800,43446.609375,45047.447229,44873.020399,44095.282851,4.190105,35493


In [14]:
seed_pred_result[2].head(10)

,y,xgboost,gboost,lightgbm,lasso,keras,average
1296,37800,36492.585938,35156.559763,35395.106946,34120.558417,6.287332,28234
140,74600,71375.390625,70741.556193,70881.065756,71771.784436,6.287332,56955
2560,74400,73961.820312,75208.859321,74841.849565,74624.284836,6.287332,59729
674,22800,22266.634766,23373.641170,23263.422322,22901.825397,6.287332,18362
118,57000,56989.925781,56964.241107,56339.702533,55066.249931,6.287332,45073
4171,77800,79498.804688,79566.546258,78052.321806,80471.418587,6.287332,63519
2125,28000,29929.000000,29497.646280,29785.364038,30705.997199,6.287332,23985
3344,81200,83816.625000,83361.074419,82487.448686,80550.410928,6.287332,66044
91,69000,65876.960938,63300.916921,64566.781764,63406.485142,6.287332,51431
3271,40400,40123.773438,41581.175297,40618.806762,41236.491264,6.287332,32713


In [15]:
seed_pred_result[3].head(10)

,y,xgboost,gboost,lightgbm,lasso,keras,average
3038,78000,75225.859375,75923.688458,74260.254962,76442.218269,8.38981,60372
953,45600,45647.875000,45825.588198,46305.120598,45266.189027,8.38981,36611
3532,105600,104901.570312,103203.714771,102184.216559,102079.475352,8.38981,82475
725,33600,33323.890625,34756.923623,34394.614771,34900.350398,8.38981,27477
3681,47600,45539.390625,45236.463293,45046.175017,46398.891236,8.38981,36446
1404,37000,38923.875000,38521.045643,38065.274995,38608.719481,8.38981,30825
1685,23200,24973.585938,24830.732141,24467.685717,24365.546440,8.38981,19729
1136,33400,34752.773438,33781.398056,33570.101404,34049.330164,8.38981,27232
1111,18400,17878.275391,17580.675640,17800.840291,15847.541326,8.38981,13823
1885,33800,34242.707031,34293.157052,33802.367889,35522.800372,8.38981,27574


In [16]:
seed_pred_result[4].head(10)

,y,xgboost,gboost,lightgbm,lasso,keras,average
1286,27600,28030.613281,28006.381529,27789.523295,26680.275387,10.482376,22103
1456,24000,22745.109375,23448.015204,23224.625279,23777.984575,10.482376,18641
2025,77800,74297.960938,76222.911815,73353.587911,75698.922104,10.482376,59917
1287,31800,26322.453125,28131.084742,27752.181246,27557.413291,10.482376,21955
3888,63000,65373.179688,66831.197260,67470.776867,66097.541470,10.482376,53157
2510,35800,36339.722656,35191.417402,35345.079407,35703.113483,10.482376,28518
2675,39600,36433.546875,35191.417402,35858.578595,35939.738815,10.482376,28687
3974,50000,46722.460938,49922.286395,48955.215162,48584.486055,10.482376,38839
1643,46400,44890.472656,43013.232882,42816.412504,43730.293555,10.482376,34892
897,28600,27376.406250,28451.295829,27912.525971,28170.882763,10.482376,22384
